In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64
from IPython.display import Image

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "1234"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('ProjectTwo')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
   # html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),style ={"height": "2" "00px"}),
    html.Center(html.B(html.H3('Tatsiana Labkovich CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    dcc.RadioItems(
    id ='filter-type',
        options=[
            {'label':'Water Rescue','value': 'WR'},
            {'label':'Mountain or Wilderness','value':'MWR'},
            {'label':'Disaster or Individual Tracking','value': 'DRIT'},
            {'label': 'Reset','value':'RESET'}
        ],
        value='RESET',
        labelStyle={'display':'inline-block'})
    
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,

        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
             

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type','value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
 if filter_type =='WR':   
        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                        {'$or': [
                                                            {'breed': 'Labrador Retriever Mix'},
                                                            {'breed': 'Chesa Bay Retr Mix'},
                                                            {'breed': 'Newfoundland Mix'},
                                                            {'breed': 'Newfoundland/Laborador Retriver'},
                                                            {'breed': 'Newfoundland/Australian Cattle Dog'},
                                                            {'breed': 'Newfoundland/Great Pyrenees'}
                                                        ]},
                                                        {'$and':[{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                {'afe_upon_outcome_in_weeks': {'$lte': 156}}]
                                                        }] })))
        if filter_type == 'MWR':
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                        {'$or': [
                                                            {'breed': 'German Sherperd'},
                                                           {'breed': 'Alaskan Malamute'},
                                                           {'breed': 'Old English Sheepdog'},
                                                           {'breed': 'Rottweiler'},
                                                           {'breed': 'Siberian Husky'},
                                                            
                                                       ]},
                                                        {'$and':[{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                               {'afe_upon_outcome_in_weeks': {'$lte': 156}}]
                                                       }]
                                                         })))
        if filter_type == 'DRIT':
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                       {'$or': [
                                                           {'breed': 'Doberman Pinscher'},
                                                            {'breed': 'German Shepherd'},
                                                            {'breed': 'Golden Retriever'},
                                                            {'breed': 'Bloodhound'},
                                                            {'breed': 'Rottweiler'},
                                                           
                                                       ]},
                                                        {'$and':[{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                                {'afe_upon_outcome_in_weeks': {'$lte': 300}}]
                                                       }]
                                                })))
            if filter_type == 'RESET':
                df = pd.DataFrame.from_records(shelter.read_all({}))
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} 
            for i in df.columns]
            data=df.to_dict('records')

            return (data,columns)

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
        return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
   Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    return [
        dcc.Graph(
        figure = px.pie(
            data_frame = dff,
            values = values,
            names = names,
            color_discreet_sequence=px.colors.sequential.RdBu,
            width = 800,
            height = 500))
  ]


      
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return[
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H2("Animal Name"),
                    html.P(dff.loc[0,'name'])
                ])
            ])
        ])
   
    ]

app

Connection successful
